<a href="https://colab.research.google.com/github/adityadhumal06/Natural_Language_Processing/blob/main/bert_email_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
spam_df = pd.read_csv('spam (2).csv')

In [3]:
spam_df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
spam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
spam_df.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [6]:
spam_df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [7]:
spam_df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [8]:
spam_df['spam'] = spam_df['Category'].apply(lambda x: 1 if x == 'spam' else 0)

In [9]:
spam_df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [10]:
x = spam_df['Message']

In [11]:
y = spam_df['spam']

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=21,stratify=y)

In [14]:
x_train.shape

(4457,)

In [15]:
y_train.shape

(4457,)

In [16]:
x_test.shape

(1115,)

In [17]:
y_test.shape

(1115,)

In [22]:
# pip install tensorflow_text


In [19]:
# pip install tensorflow_hub

In [23]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import time

In [24]:
# bert preprocess text
bert_preprocess = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")


# bert_encoder
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-12-h-768-a-12/versions/2")


In [25]:
def get_sentence_embedding(sentence):
  preprocessed_text = bert_preprocess(sentence)

  return bert_encoder(preprocessed_text)['pooled_output']


In [26]:
get_sentence_embedding([
    'Ok lar... Joking wif u oni...',
    "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.30529007, -0.5115735 ,  0.78161347, ..., -0.99944913,
         0.5490462 ,  0.98765856],
       [-0.25152332, -0.27989385,  0.4470586 , ..., -0.9706896 ,
         0.2701109 ,  0.9864019 ]], dtype=float32)>

In [27]:
e = get_sentence_embedding([
    'banana',
    'grapes',
    'aditya',
    'jeff bezos',
    'virat kohli'
])
print(e)

tf.Tensor(
[[ 0.26732677 -0.19118229  0.08356225 ... -0.6183356   0.00336342
   0.999784  ]
 [ 0.36010408  0.14107582  0.91148835 ... -0.8994039  -0.11671514
   0.9992937 ]
 [ 0.21503474 -0.27579242  0.46329388 ... -0.8497097   0.5036511
   0.9876441 ]
 [ 0.3807956   0.2854638   0.9919518  ... -0.7380235   0.09374462
   0.99240774]
 [-0.10102995  0.02693401  0.5384349  ... -0.9898081   0.74866503
   0.9900768 ]], shape=(5, 768), dtype=float32)


In [29]:
from sklearn.metrics.pairwise import cosine_similarity


In [30]:
cosine_similarity([e[4]],[e[3]])

array([[0.66921955]], dtype=float32)

In [31]:
cosine_similarity([e[0]],[e[3]])

array([[0.03427694]], dtype=float32)

In [32]:
text_input = tf.keras.layers.Input(shape=(), dtype= tf.string, name='text')
preprocess_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocess_text)
outputs['pooled_output']

l = tf.keras.layers.Dropout(0.3,name='Dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

model = tf.keras.Model(inputs=[text_input],outputs=[l])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_word_ids': (None,                                           
                              128),                                                               
                              'input_type_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [33]:
model.compile(
    optimizer='adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [35]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
140/140 [==============================] - 46s 329ms/step - loss: 0.1252 - accuracy: 0.9592
Epoch 2/5
140/140 [==============================] - 45s 324ms/step - loss: 0.1069 - accuracy: 0.9684
Epoch 3/5
140/140 [==============================] - 46s 329ms/step - loss: 0.0980 - accuracy: 0.9697
Epoch 4/5
140/140 [==============================] - 46s 326ms/step - loss: 0.0898 - accuracy: 0.9742
Epoch 5/5
140/140 [==============================] - 45s 323ms/step - loss: 0.0846 - accuracy: 0.9742


In [41]:
model.evaluate(x_test,y_test)

35/35 [==============================] - 12s 323ms/step - loss: 0.0698 - accuracy: 0.9785


[0.06979170441627502, 0.9784753322601318]

In [53]:
reviews = ['Get rich quick! Earn $$$ fast!',
           'Hey, how are you doing?',
           'Claim your prize now!'
           'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
           "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]

In [54]:
type(reviews)

list

In [55]:
model.predict(reviews)

1/1 [==============================] - 0s 44ms/step


array([[0.47742698],
       [0.00116097],
       [0.7311783 ]], dtype=float32)

In [52]:
spam_df['Message'][2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"